# Publications markdown generator for academicpages

Takes a TSV of publications with metadata and converts them for use with [academicpages.github.io](academicpages.github.io). This is an interactive Jupyter notebook ([see more info here](http://jupyter-notebook-beginner-guide.readthedocs.io/en/latest/what_is_jupyter.html)). The core python code is also in `publications.py`. Run either from the `markdown_generator` folder after replacing `publications.tsv` with one containing your data.

TODO: Make this work with BibTex and other databases of citations, rather than Stuart's non-standard TSV format and citation style.


## Data format

The TSV needs to have the following columns: proj-date, title, excerpt, source, url_slug, project_url, and code_url, with a header at the top. 
- `excerpt` can be blank, but the others must have values. 
- `proj-date` must be formatted as YYYY-MM-DD.
- `url_slug` will be the descriptive part of the .md file and the permalink URL for the page about the paper. The .md file will be `YYYY-MM-DD-[url_slug].md` and the permalink will be `https://[yourdomain]/publications/YYYY-MM-DD-[url_slug]`

This is how the raw file looks (it doesn't look pretty, use a spreadsheet or other program to edit and create).

In [6]:
!cat projects.tsv

proj_date	title	excerpt	source	url_slug	project_url    code_url
2021-05-31	Snoring Detection on a Microcontroller	Snoring is related to a common medical condition that can lead to many serious health issues including diabetes, stroke, and depression. Due to the negative health impacts of such medical condition, it is crucial for people to know whether they snore and understand their snoring patterns and triggers of snoring. In this paper, we propose a bed-side snoring detection program on microcontroller device that automatically identifies snoring sounds.This paper explores the development of a testing methodology in the context of accurately predicting future battery performance using machine learning. In particular, I propose a Dense Neural Network Model that can predict battery life using data from the first 50 cycles in the battery life.	Rotaru, A., Kelly Zhang, Jiayu Yao (2020). "Snoring Detection on a Microcontroller" <i>Harvard CS249r: Tiny Machine Learning (TinyML) </i>. 1(1).

## Import pandas

We are using the very handy pandas library for dataframes.

In [7]:
# ! pip install -U pandas
import pandas as pd

## Import TSV

Pandas makes this easy with the read_csv function. We are using a TSV, so we specify the separator as a tab, or `\t`.

I found it important to put this data in a tab-separated values format, because there are a lot of commas in this kind of data and comma-separated values can get messed up. However, you can modify the import statement, as pandas also has read_excel(), read_json(), and others.

In [14]:
publications = pd.read_csv("projects.tsv", sep="\t", header=0)
publications


,proj_date,title,excerpt,source,url_slug,project_url,code_url
0,2021-05-31,Snoring Detection on a Microcontroller,Snoring is related to a common medical conditi...,"Rotaru, A., Kelly Zhang, Jiayu Yao (2020). ""Sn...",snoring-detection,https://www.youtube.com/watch?v=aQpIooBEGsA,https://github.com/adrianagaler/Snoring-Detection


## Escape special characters

YAML is very picky about how it takes a valid string, so we are replacing single and double quotes (and ampersands) with their HTML encoded equivilents. This makes them look not so readable in raw format, but they are parsed and rendered nicely.

In [15]:
html_escape_table = {
    "&": "&amp;",
    '"': "&quot;",
    "'": "&apos;"
    }

def html_escape(text):
    """Produce entities within text."""
    return "".join(html_escape_table.get(c,c) for c in text)

## Creating the markdown files

This is where the heavy lifting is done. This loops through all the rows in the TSV dataframe, then starts to concatentate a big string (```md```) that contains the markdown for each type. It does the YAML metadata first, then does the description for the individual page.

In [16]:
import os
for row, item in publications.iterrows():
    
    md_filename = str(item.proj_date) + "-" + item.url_slug + ".md"
    html_filename = str(item.proj_date) + "-" + item.url_slug
    year = item.proj_date[:4]
    
    ## YAML variables
    
    md = "---\ntitle: \""   + item.title + '"\n'
    
    md += """collection: projects"""
    
    md += """\npermalink: /project/""" + html_filename
    
    if len(str(item.excerpt)) > 5:
        md += "\nexcerpt: '" + html_escape(item.excerpt) + "'"
    
    md += "\ndate: " + str(item.proj_date) 
    
    md += "\nprojecturl: '" + html_escape(item.project_url) + "'"
    
    if len(str(item.project_url)) > 5:
        md += "\nprojecturl: '" + item.project_url + "'"
    
    md += "\ncodeurl: '" + html_escape(item.code_url) + "'"
    
    if len(str(item.code_url)) > 5:
        md += "\ncodeurl: '" + item.code_url + "'"
    
    md += "\nsource: '" + html_escape(item.source) + "'"
    
    md += "\n---"
    
    ## Markdown description for individual page
        
    if len(str(item.excerpt)) > 5:
        md += "\n" + html_escape(item.excerpt) + "\n"
    
    if len(str(item.code_url)) > 5:
        md += "\n[Access code here](" + item.code_url + ")\n" 
        
    
    md_filename = os.path.basename(md_filename)
       
    with open("../_projects/" + md_filename, 'w') as f:
        f.write(md)

AttributeError: 'Series' object has no attribute 'paper_url'

These files are in the publications directory, one directory below where we're working from.

In [15]:
!ls ../_publications/

2010-10-01-paper-title-number-2.md 2021-05-31-embedded-dnn-battery.md
2015-10-01-paper-title-number-3.md


In [16]:
!cat ../_publications/2009-10-01-paper-title-number-1.md

cat: ../_publications/2009-10-01-paper-title-number-1.md: No such file or directory
